In [1]:
import numpy as np
import tensorflow

In [2]:
from tensorflow import keras

In [5]:
aa = {}
aa.get??

In [9]:
class Vocab(object):
    def __init__(self):
        self.rev_vocabulary_mapping = []  ## (V,E)
        self.vocabulary_mapping = {}  ## каждому слову индекс строки из матрицы E
        self.embeddings = None
    
    def unvectorize(self, ids):
        return ' '.join([self.rev_vocabulary_mapping[i] for i in ids])
    
    def vectorize(self, tokens):
        return [self.vocabulary_mapping.get(w, self.vocabulary_mapping['<UNK>']) for w in tokens]
    
    def load_embeddings(self, filename, embedding_size=50, cut_at=400000):
        # список векторов
        embeddings_list = []
        # словарь слово-индекс
        self.vocabulary_mapping = {'<PAD>': 0} # занесём ключ соотетствующий отступу
        self.rev_vocabulary_mapping.append('<PAD>')
        pad = np.zeros(embedding_size) # создадим вектор для PAD
        embeddings_list.append(pad)

        with open(filename,encoding='utf8') as glove_file: 
            i = 1
            for line in glove_file:
                fields = line.split()
                token = fields[0]
                vector = np.array(list(map(float, fields[1:])))
                
                embeddings_list.append(vector)
                self.vocabulary_mapping[token] = i
                self.rev_vocabulary_mapping.append(token)
                i += 1
                if i >= cut_at:
                    break
                    
        self.vocabulary_mapping['<UNK>'] = len(embeddings_list)
        self.rev_vocabulary_mapping.append('<UNK>')
        unk = np.mean(embeddings_list[1:], axis=0) # считаем средний вектор
        embeddings_list.append(unk)

        self.embeddings = np.array(embeddings_list)
                
        print(f'Loaded {self.embeddings.shape} embeddings')

In [10]:
vocab = Vocab()
vocab.load_embeddings('glove.6B.50d.txt')

Loaded (400001, 50) embeddings


In [17]:
v = vocab.vectorize(['mother','of','god'])
v

[809, 4, 1534]

In [18]:
vocab.unvectorize(v)

'mother of god'

### Загрузим ATIS

In [20]:
def load_csv(filename):
    results = []
    with open(filename) as f:
        for l in f:
            results.append(l.strip())
            
    return results

In [27]:
slots = load_csv('data/atis.dict.slots.csv')
intents = load_csv('data/atis.dict.intent.csv')
atis_vocab = load_csv('data/atis.dict.vocab.csv')

In [29]:
atis_vocab

["'d",
 "'hare",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'t",
 "'ve",
 '0900',
 '1',
 '10',
 '100',
 '1000',
 '1017',
 '1020',
 '1024',
 '1026',
 '1030',
 '1039',
 '1045',
 '1055',
 '1059',
 '106',
 '1083',
 '11',
 '110',
 '1100',
 '1110',
 '1115',
 '1130',
 '1133',
 '1145',
 '1158',
 '12',
 '1200',
 '1201',
 '1205',
 '1207',
 '1209',
 '1220',
 '1222',
 '1230',
 '124',
 '1245',
 '1288',
 '1291',
 '130',
 '1300',
 '137338',
 '139',
 '150',
 '1500',
 '1505',
 '1600',
 '163',
 '1700',
 '1765',
 '1800',
 '1850',
 '19',
 '1940',
 '1991',
 '1992',
 '1993',
 '1994',
 '2',
 '20',
 '200',
 '201',
 '21',
 '210',
 '2100',
 '212',
 '2134',
 '2153',
 '217',
 '225',
 '229',
 '230',
 '257',
 '269',
 '270',
 '271',
 '279',
 '281',
 '296',
 '297',
 '3',
 '300',
 '305',
 '311',
 '315',
 '320',
 '323',
 '324',
 '329',
 '3357',
 '343',
 '345',
 '352',
 '3724',
 '382',
 '4',
 '400',
 '402',
 '405',
 '415',
 '416',
 '417',
 '419',
 '420',
 '428',
 '43',
 '430',
 '4400',
 '445',
 '459',
 '466',
 '468',
 '486',
 '497

In [72]:
atis_train_queries = load_csv('data/atis.train.query.csv')
atis_train_intents = load_csv('data/atis.train.intent.csv')
atis_train_slots = load_csv('data/atis.train.slots.csv')

In [73]:
def atis_ids_to_words(ids, atis_vocab):
    return [atis_vocab[i] for i in ids]


In [74]:
atis_train_queries = [[int(index) for index in line.split()] for line in atis_train_queries]
atis_train_intents = [int(index) for index in atis_train_intents]
atis_train_slots = [[int(index) for index in line.split()] for line in atis_train_slots]

In [46]:
[' '.join([atis_vocab[i] for i in line]) for line in atis_train_queries]

['BOS i want to fly from boston at 838 am and arrive in denver at 1110 in the morning EOS',
 'BOS what flights are available from pittsburgh to baltimore on thursday morning EOS',
 'BOS what is the arrival time in san francisco for the 755 am flight leaving washington EOS',
 'BOS cheapest airfare from tacoma to orlando EOS',
 'BOS round trip fares from pittsburgh to philadelphia under 1000 dollars EOS',
 'BOS i need a flight tomorrow from columbus to minneapolis EOS',
 'BOS what kind of aircraft is used on a flight from cleveland to dallas EOS',
 'BOS show me the flights from pittsburgh to los angeles on thursday EOS',
 'BOS all flights from boston to washington EOS',
 'BOS what kind of ground transportation is available in denver EOS',
 'BOS show me the flights from dallas to san francisco EOS',
 'BOS show me the flights from san diego to newark by way of houston EOS',
 "BOS what 's the airport at orlando EOS",
 'BOS what is the cheapest flight from boston to bwi EOS',
 'BOS all fligh

In [49]:
atis_train_queries_w = [[atis_vocab[i] for i in line] for line in atis_train_queries]

In [52]:
train_sent = [vocab.vectorize(tokens[1:-1]) for tokens in atis_train_queries_w]

In [64]:
def build_classification_model(embeddings):
    # Точка входа в граф задаётся при помощи специальных тензоров типа Input
    # Первая координата соответсвует длине текста, так как тексты в датасете имеют разную длину
    # значение считается переменным
    text_input = keras.layers.Input(shape=(None,), dtype='int32') ## batch, num sentence = None, last = 1 - индексы из glove
    
    # Создаём специальный слой для работы с embedding, 
    # Его функция -- заменять индентификатор вектором из Glove
    # Указываем trainable = False, чтобы векторы embedding'ов не изменялись в процессе обучения
    embedding_layer = keras.layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       trainable = False) ## хорошая практика учить с замороженными embedding, а потом их разморозить
    
    x = embedding_layer(text_input)
    
    # Создаём рекуррентную ячейку
    # Первый параметр отвечает за размер внутреннего состояния (памяти ячейки)
    # По умолчанию такой слой возвращает только последнее состояние (см. картинку),
    # Если мы хотим получить состояния на каждом шаге необходимо указать return_sequences = True
    x = keras.layers.LSTM(128, recurrent_dropout=0.25)(x)
    
    # Полученный результат направляем в полносвязный слой, который будет осуществлять классификацию
    output = keras.layers.Dense(26, activation='softmax')(x)
    
    model = keras.models.Model(inputs=[text_input], outputs=[output], name = 'LSTM_classifier')
    
    return model

In [65]:
keras.backend.clear_session()
model = build_classification_model(vocab.embeddings)

In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               91648     
_________________________________________________________________
dense (Dense)                (None, 26)                3354      
Total params: 20,095,052
Trainable params: 95,002
Non-trainable params: 20,000,050
_________________________________________________________________


In [67]:
adam = keras.optimizers.Adam(lr=0.0001)

model.compile(adam, keras.losses.sparse_categorical_crossentropy, metrics = ['acc'])

In [77]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(atis_train_queries, atis_train_intents, test_size = 0.1)

In [78]:
def generate_batches(x,y,batch_size = 64):
    i=0
    while True:
        i = i %len(x)
        yield keras.preprocessing.sequence.pad_sequences(x[i:i+batch_size]),y[i:i+batch_size]
        i += batch_size

In [80]:
train_generator = generate_batches(x_train, y_train)
val_generator = generate_batches(x_val,y_val)